In [1]:
data_dir = "/home/vagrant/taxi2popularity/theatre/data/"

In [2]:
import dill
dat = dill.load(open(data_dir + 'all_theatre_scraped2009_to_2017.pkd', 'r'))

In [5]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

In [6]:
output_notebook()

Loading BokehJS ...

In [9]:
from dateutil.parser import parse as date_parse
timeline = sorted(map(date_parse, dat.keys()))

In [11]:
timeline[0].format

AttributeError: 'datetime.datetime' object has no attribute 'format'

In [40]:
for date_ in timeline:
    datestr = date_.date().isoformat()
    tmp = dat[datestr]
    dat[datestr] = [clean_theatre_doc(row) for row in tmp]

In [41]:
import dill
dill.dump(dat, open(data_dir+'all_theatre_scraped2009_to_2017.dill', 'w'))

In [42]:
def clean_var(st):
    if (type(st) is not unicode) and (type(st) is not str):
        return st
    st2=st.replace("%","").replace("$","").replace(",","")
    try:
        tmp = float(st2)
    except Exception as e:
        return st
    if abs(tmp%1) < .00000001:
        return int(tmp)
    return tmp

def clean_theatre_doc(doc):
    for k,v in doc.items():
        if k in ['Theatre', 'Show']:
            continue
        doc[k] = clean_var(v)
    return doc

def get_box_office(date_, theatre="AMBASSADOR"):
    datestr = date_.date().isoformat()
    for row in dat[datestr]:
        if row['Theatre']==theatre:
            return row # clean_theatre_doc(row)
    return {'SeatsSold':0}

In [50]:
get_box_office(timeline[-10], 'MAJESTIC')

{u'Avg.Paid': 82.76,
 u'Diff $': -75695,
 u'Diff %': -2.5,
 u'Gross % Potential': 50,
 u'Last Week': 72.8,
 u'Last Week Gross': 822845,
 u'Per': 8,
 u'Potential Gross': 1493252,
 u'SeatsSold': 9028,
 u'Show': 'THE PHANTOM OF THE OPERA',
 'Theatre': 'MAJESTIC',
 u'This Week': 70.3,
 u'Top Ticket': 203,
 u'TotalSeats': 12840}

In [51]:
# weekly ticket sales for Ambassador Theatre
from bokeh.charts import TimeSeries
data = {'Date':timeline}
for theatre in ['MAJESTIC', 'AMBASSADOR', 'RICHARD RODGERS', 'GERSHWIN']:
    data[theatre]=map(lambda t: get_box_office(t,theatre)['SeatsSold'], timeline)

In [52]:
p = TimeSeries(data, x='Date', y=['MAJESTIC', 'AMBASSADOR', 'RICHARD RODGERS','GERSHWIN'], title="MAJESTIC Box office", ylabel='Seats Sold')
t =show(p, notebook_handle=True)